# Using the generated SDK and data models

## Preparation

The main entry point for the SDK is the `PowerOpsClient`

To get started, we assume you have installed the `cognite-power-ops` SDK and that all configurations have been setup in Cognite Data Fusion (CDF).

Furthermore, it is assumed that you have setup one or two `toml` files with the credentials for connecting to Cognite Data Fusion and settings for this SDK.


### Interactive Login
This only requires one `toml` file looking like this

`settings.toml`

```toml
[cognite]
  login_flow = "interactive"
  project = "<cdf-project>"
  tenant_id = "<tenant-id>"
  cdf_cluster = "<cdf-cluster>"
  client_id = "<client-id>"

[powerops]
  read_dataset = "uc:000:powerops"
  write_dataset = "uc:000:powerops"
  monitor_dataset = "uc:po:monitoring"

```

### Client Credentials Login

For this case the you can use two toml files to separate the secrets from the regular settings.


`settings.toml`

```toml
[cognite]
  login_flow = "client_credentials"
  project = "<cdf-project>"
  tenant_id = "<tenant-id>"
  cdf_cluster = "<cdf-cluster>"
  client_id = "<client-id>"

[powerops]
  read_dataset = "uc:000:powerops"
  write_dataset = "uc:000:powerops"
  monitor_dataset = "uc:po:monitoring"
```
and the `.secrets.toml`

```toml
[cognite]
  client_secret = "<client-secret>"

```

Values in `.secrets.toml` will overwrite those in `settings.toml`




In [15]:
# You can control which setting files are loaded through the environmental variable below.
# In this case, the setting files are located two levels above, in the root of the repository.
import os
from cognite.powerops._version import __version__
from cognite.powerops import PowerOpsClient


os.environ["SETTINGS_FILES"] = "../../settings.toml;../../.secrets.toml"

powerops = PowerOpsClient.from_settings()

WRITE_DATA_SET = powerops.datasets.write_dataset_id

print(__version__)  # Print the version of the package

0.101.1



## Setting up a ShopCase instance that can be triggered:
1. Upload a (set of) file(s) for a shop case
2. Set up a ShopCase, connecting them with a scenario
3. Write the ShopCase instance. Verify that it was created using the SDK
4. Trigger a shop execution of that shop case


### Helpers for all types of case generation

In [2]:
import datetime
from pathlib import Path
from cognite.powerops.client._generated.v1.data_classes import (
    # These data classes are used to send data to CDF
    # There are non-write versions of these classes as well and they are used to read data from CDF
    ShopCaseWrite,  # This model contains everything needed to execute a SHOP run
    ShopFileWrite,  # A container that holds a reference to a file in CDF
    ShopModelWrite,  # A static model in a file reference + references to time series data
    ShopScenarioWrite,  # A way to modify run configurations for a given model
)
from cognite.powerops.client._generated.v1.data_classes._shop_case import ShopCase


# Directory where the example case files are located
EXAMPLE_CASES_ROOT = Path("example_case_files")


# helper method for all case generations
def upload_file(
    file_name: str,
    external_id: str | None = None,
    mime_type: str = "application/yaml",
) -> str:
    """For simplicity we we will use the file name as the external id,
    but it can be can use any unique string as the external id."""
    external_id = external_id or file_name
    file_path = (EXAMPLE_CASES_ROOT / file_name).resolve()
    file = powerops.cdf.files.upload(
        path=str(file_path),
        external_id=external_id,
        name=file_name,
        data_set_id=WRITE_DATA_SET,
        mime_type=mime_type,
        # Overwrite the file at the given external is if it already exists
        # This will also overwrite potentially existing metadata
        overwrite=True,
    )
    return file.external_id


def retrieve_shop_case(shop_case_external_id: str) -> ShopCase:
    """Retrieve a shop case based on external id"""
    # NB: Sometimes this can return None even if the case exists if it was just created
    # An alterative way to check if the case is by using the cognite data modeling API directly.
    # Read the Cognitive Data Modeling documentation for more information.
    return powerops.v1.day_ahead_bid.shop_case.retrieve(shop_case_external_id)

### Example A: Complete case file

In this case, the `ShopScenario` as and its `ShopModel` are mostly superfluous. 
However, they are still added as nearly empty objects in order to set the SHOP version 


In [3]:
EXAMPLE_CASE_STAVANGER = "example_case_stavanger"


def create_shop_case_stavanger(shop_version: str = "15.6.1.0") -> ShopCaseWrite:
    # NB! This is a minimal example of a shop case -- 
    file_reference = upload_file(file_name="a_example_stavanger_with_commands.yaml")

    shop_case_write = ShopCaseWrite(
        externalId=EXAMPLE_CASE_STAVANGER,  # unique identifier for the case, used to trigger SHOP execution
        # The time range SHOP is optimized over
        startTime=datetime.datetime(2024, 5, 31, 22),
        endTime=datetime.datetime(2024, 6, 2, 22),
        # The scenario is used to modify the run configuration of the model
        scenario=ShopScenarioWrite(
            name="a_dummy_scenario",
            # The model is a static model in a file reference + references to time series data
            model=ShopModelWrite(
                name="a_dummy_model",
                shop_version=shop_version,
            ),
        ),
        # The files that are used in the case
        shopFiles=[
            ShopFileWrite(
                name="a_case_stavanger_with_commands",
                label="",
                fileReference=file_reference,  # external id of the file on CDF
                isAscii=False,
                order=1,
            ),
        ],
    )
    return shop_case_write


case_stavanger_write = create_shop_case_stavanger()

In [4]:
# Upload the case to CDF, using the WriteObject created above
powerops.v1.upsert(case_stavanger_write, replace=True, allow_version_increase=False)

ResourcesWriteResult(nodes=[<NodeApplyResult(space='power_ops_instances', external_id='example_case_stavanger', version=1) at 0x7fb3c1b6eb90>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:62054b74b3db4b7a9398c87c9ae6c445', version=1) at 0x7fb3c099f790>, <NodeApplyResult(space='power_ops_instances', external_id='shopscenario:a08e5c7542ee4b0a8e8687eb13f566d9', version=1) at 0x7fb3c1b410d0>, <NodeApplyResult(space='power_ops_instances', external_id='shopmodel:fd0a4d9368294b18a1d8b84a8cb3d558', version=1) at 0x7fb418340490>], edges=[<EdgeApplyResult(space='power_ops_instances', external_id='example_case_stavanger:shopfile:62054b74b3db4b7a9398c87c9ae6c445', version=1) at 0x7fb3c1b37a50>], time_series=[])

In [5]:
retrieve_shop_case(EXAMPLE_CASE_STAVANGER)

,value
space,power_ops_instances
external_id,example_case_stavanger
data_record,"{'version': 1, 'last_updated_time': 2024-10-21..."
node_type,None
scenario,shopscenario:a08e5c7542ee4b0a8e8687eb13f566d9
start_time,2024-05-31 22:00:00+00:00
end_time,2024-06-02 22:00:00+00:00
shop_files,[shopfile:62054b74b3db4b7a9398c87c9ae6c445]


In [6]:
# Trigger the case
powerops.cogshop.trigger_shop_case(case_stavanger_write.external_id)

### Example B: Almost complete case file

In this case, the `ShopScenario` as and its `ShopModel` are mostly superfluous. 
However, they are still added as nearly empty objects in order to set the SHOP version

A change here is that we extend the list in `ShopFile` and we need to specify the order that the files should be loaded into SHOP. 

Notice the label on the commands file. This is necessary as CogSHOP expects the commands to be labeled.  


In [7]:
EXAMPLE_CASE_FORNEBU = "example_case_fornebu"


def create_shop_case_fornebu(shop_version: str = "16.0.2") -> ShopCaseWrite:
    # NB! this case does not have any cut groups, shop result is not reasonable
    file_reference_case = upload_file(file_name="b_example_fornebu_without_commands.yaml")
    file_reference_commands = upload_file(file_name="b_commands.yaml")

    shop_case_write = ShopCaseWrite(
        externalId=EXAMPLE_CASE_FORNEBU,  # unique identifier for the case, used to trigger SHOP execution
        # The time range SHOP is optimized over
        startTime=datetime.datetime(2023, 1, 19, 23),
        endTime=datetime.datetime(2023, 1, 29, 23),
        # The scenario is used to modify the run configuration of the model
        scenario=ShopScenarioWrite(
            name="b_dummy_scenario",
            # The model is a static model in a file reference + references to time series data
            model=ShopModelWrite(
                name="b_dummy_model",
                shop_version=shop_version,
            ),
        ),
        # The files that are used in the case
        shopFiles=[
            ShopFileWrite(
                name="b_case_fornebu_no_commands",
                label="",
                fileReference=file_reference_case,  # external id of the file on CDF
                isAscii=False,
                order=1,
            ),
            ShopFileWrite(
                name="b_case_fornebu_commands",
                label="commands",
                fileReference=file_reference_commands,  # external id of the file on CDF
                isAscii=False,
                order=2,
            ),
        ],
    )
    return shop_case_write


case_fornebu_write = create_shop_case_fornebu()

In [8]:
# Upload the case to CDF, using the WriteObject created above
powerops.v1.upsert(case_fornebu_write, replace=True, allow_version_increase=False)

ResourcesWriteResult(nodes=[<NodeApplyResult(space='power_ops_instances', external_id='example_case_fornebu', version=1) at 0x7fb3c0a17810>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:b89b718bfd3e440190a1283774614fad', version=1) at 0x7fb3c0a17a50>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:b07dd181892248279b26ac75455f461c', version=1) at 0x7fb3c1b67350>, <NodeApplyResult(space='power_ops_instances', external_id='shopscenario:bc1c9aedc34e4fd1a33afea94cc48049', version=1) at 0x7fb3c1b65810>, <NodeApplyResult(space='power_ops_instances', external_id='shopmodel:5d3ba39f54aa4d1186a3489eab741b61', version=1) at 0x7fb3c1b667d0>], edges=[<EdgeApplyResult(space='power_ops_instances', external_id='example_case_fornebu:shopfile:b89b718bfd3e440190a1283774614fad', version=1) at 0x7fb3c09d4a50>, <EdgeApplyResult(space='power_ops_instances', external_id='example_case_fornebu:shopfile:b07dd181892248279b26ac75455f461c', version=1) at 0x7fb3c09d7690>],

In [9]:
retrieve_shop_case(EXAMPLE_CASE_FORNEBU)

,value
space,power_ops_instances
external_id,example_case_fornebu
data_record,"{'version': 1, 'last_updated_time': 2024-10-21..."
node_type,None
scenario,shopscenario:bc1c9aedc34e4fd1a33afea94cc48049
start_time,2023-01-19 23:00:00+00:00
end_time,2023-01-29 23:00:00+00:00
shop_files,"[shopfile:b89b718bfd3e440190a1283774614fad, sh..."


In [10]:
# Trigger the case
powerops.cogshop.trigger_shop_case(case_fornebu_write.external_id)

## Using the generated SDK to view a ShopResult instance

1. Querying for ShopResult instance, given their ShopCase external id
2. Using the external ID of a ShopResult to retrieve and inspect it  

In [11]:
from cognite.client import data_modeling as dm
from cognite.powerops.client._generated.v1.data_classes import ShopResultList


# Query the shop results based on the external id of the shop case
def result_instance_query(
    shop_case_external_id: str | list[str],
    external_id_prefix: str | None = None,
    limit: int | None = None,
    filter: dm.Filter | None = None,
) -> ShopResultList:
    """
    Query the shop results based on the external id of the shop case

    Parameters:
    shop_case_external_id: str | list[str]
        External id(s) of the shop cases that were triggered
    external_id_prefix: str | None
        Prefix of the external id(s)
    limit: int | None
        Maximum number of results to return. Defaults to 3
    filter: dm.Filter | None
        For advanced filtering in the case that the other parameters insufficient
        Refer to the Cognite Data Modeling documentation for more details

    Returns:
        ShopResultList

    """
    # Generate the query
    call_query = powerops.v1.day_ahead_bid.shop_result(
        case=shop_case_external_id,
        external_id_prefix=external_id_prefix,
        limit=limit,
        filter=filter,
    )
    # Execute and return the result
    return call_query.query()

In [18]:
# We convert the result to a pandas dataframe for easier data handling
all_results_df_1 = result_instance_query(
    shop_case_external_id=[EXAMPLE_CASE_STAVANGER, EXAMPLE_CASE_FORNEBU], limit=5
).to_pandas()

The (some) of the columns of the dataframe: 

* External id: The external id of the instance
* Case: The external id of the shop case that the instance belongs to
* Objective value: The objective value of the Shop run
* Pre run: A file reference to the pre run yaml file
* Post run: A file reference to the post run yaml file
* Messages: A file reference to the logs generated by Shop
* Cplex logs: A file reference to the cplex logs generated by Shop
* Data record: The data record of the instance, contains `last_updated_time` and `created_time`


In [19]:
all_results_df_1

,space,external_id,case,objective_value,pre_run,post_run,messages,cplex_logs,alerts,output_time_series,node_type,data_record
0,power_ops_instances,shop_result__dfd0ed6a-d975-4296-9b75-0574394e15f4,example_case_stavanger,"{'total': -380036.7316799998, 'load_value': 0....",a_example_stavanger_with_commands.yaml,POWEROPS_SHOP_post-run-dfd0ed6a-d975-4296-9b75...,POWEROPS_SHOP_shop-dfd0ed6a-d975-4296-9b75-057...,POWEROPS_SHOP_cplex-dfd0ed6a-d975-4296-9b75-05...,None,None,None,"{'version': 1, 'last_updated_time': 2024-10-21..."
1,power_ops_instances,shop_result__294f124d-0546-40b4-94fd-4d2a40194f32,example_case_fornebu,"{'total': -19927822739.279236, 'load_value': 0...",POWEROPS_SHOP_pre-run-294f124d-0546-40b4-94fd-...,POWEROPS_SHOP_post-run-294f124d-0546-40b4-94fd...,POWEROPS_SHOP_shop-294f124d-0546-40b4-94fd-4d2...,POWEROPS_SHOP_cplex-294f124d-0546-40b4-94fd-4d...,None,None,None,"{'version': 1, 'last_updated_time': 2024-10-21..."


In [15]:
import pandas as pd
from cognite.powerops.client._generated.v1.data_classes import (
    ShopResult,
    ShopResultList,
)


# Retrieve a specific shop result instance based on the external id of the shop result
def result_instance_retrieve(external_id: str | list[str]) -> ShopResultList:
    """Returns a list of ShopResult instances based on the external id of the shop result"""
    instance = powerops.v1.shop_based_day_ahead_bid_process.shop_result.retrieve(
        external_id=external_id
    )
    return instance

In [20]:
# Get the row where the case is EXAMPLE_CASE_A
case_stavanger_result: pd.DataFrame = all_results_df_1.loc[
    all_results_df_1["case"] == EXAMPLE_CASE_STAVANGER
]
# Retrieve the first result instance from the dataframe above. This guarantees that the result exists
shop_result_stavanger: ShopResult = result_instance_retrieve(case_stavanger_result.external_id)[0]
shop_result_stavanger

,value
space,power_ops_instances
external_id,shop_result__dfd0ed6a-d975-4296-9b75-0574394e15f4
data_record,"{'version': 1, 'last_updated_time': 2024-10-21..."
node_type,None
case,example_case_stavanger
objective_value,"{'total': -380036.7316799998, 'load_value': 0...."
pre_run,a_example_stavanger_with_commands.yaml
post_run,POWEROPS_SHOP_post-run-dfd0ed6a-d975-4296-9b75...
messages,POWEROPS_SHOP_shop-dfd0ed6a-d975-4296-9b75-057...
cplex_logs,POWEROPS_SHOP_cplex-dfd0ed6a-d975-4296-9b75-05...


### Looking at a file referenced by the ShopResult.


In [18]:
from tempfile import TemporaryDirectory


def download_shop_messages(shop_result: ShopResult):
    """Download a file reference from ShopResult to a temporary directory and print the first 10 lines"""
    with TemporaryDirectory() as temp_dir:
        file_path = Path(temp_dir) / "shop_result_file"
        powerops.cdf.files.download_to_path(
            path=file_path,
            external_id=shop_result.messages,  # Or `.pre_run`, `.post_run`, `.cplex_logs`
        )
        # Print the first 10 lines of the file
        with open(file_path, "r") as file:
            for i, line in enumerate(file):
                print(line)
                if i >= 15:
                    break


download_shop_messages(shop_result_stavanger)

INFORMATION: 1032

15.6.1.0 Cplex 20.1.0 Gurobi 7.5 OSI/CBC 2.9 2024-05-10



INFORMATION: 1047

Current time: Mon Oct 21 11:02:04 2024



DIAGNOSIS WARNING: 3215

Plant Sola, generating unit 1: No generator data

DIAGNOSIS WARNING: 3215

Plant Sola, generating unit 2: No generator data

DIAGNOSIS WARNING: 3205

Plant Sola: 0 errors 2 warnings.

--------------*----------------*------------


Model testing gave 2 warnings.

--------------*----------------*------------



### More than one run of one case

In the case that a case was triggered multiple times, several `ShopResult`-instances will exist with a link with the same `case` link.

We can figure out which result was the most recently generated by looking at the `data_record` column. 

We will re-trigger another run of `EXAMPLE_CASE_FORNEBU` to demonstrate and then compare two results that are linked to it.

In [26]:
# Trigger example case Fornebu again
powerops.cogshop.trigger_shop_case(EXAMPLE_CASE_FORNEBU)

In [36]:
# We convert the result to a pandas dataframe for easier data handling
all_results_df_2 = result_instance_query(
    shop_case_external_id=[EXAMPLE_CASE_STAVANGER, EXAMPLE_CASE_FORNEBU], limit=5
).to_pandas()

In [37]:
all_results_df_2

,space,external_id,case,objective_value,pre_run,post_run,messages,cplex_logs,alerts,output_time_series,node_type,data_record
0,power_ops_instances,shop_result__dfd0ed6a-d975-4296-9b75-0574394e15f4,example_case_stavanger,"{'total': -380036.7316799998, 'load_value': 0....",a_example_stavanger_with_commands.yaml,POWEROPS_SHOP_post-run-dfd0ed6a-d975-4296-9b75...,POWEROPS_SHOP_shop-dfd0ed6a-d975-4296-9b75-057...,POWEROPS_SHOP_cplex-dfd0ed6a-d975-4296-9b75-05...,None,None,None,"{'version': 1, 'last_updated_time': 2024-10-21..."
1,power_ops_instances,shop_result__294f124d-0546-40b4-94fd-4d2a40194f32,example_case_fornebu,"{'total': -19927822739.279236, 'load_value': 0...",POWEROPS_SHOP_pre-run-294f124d-0546-40b4-94fd-...,POWEROPS_SHOP_post-run-294f124d-0546-40b4-94fd...,POWEROPS_SHOP_shop-294f124d-0546-40b4-94fd-4d2...,POWEROPS_SHOP_cplex-294f124d-0546-40b4-94fd-4d...,None,None,None,"{'version': 1, 'last_updated_time': 2024-10-21..."
2,power_ops_instances,shop_result__abe63578-86c8-43f0-b1c5-6143182916d2,example_case_fornebu,"{'total': -19927822739.279236, 'load_value': 0...",POWEROPS_SHOP_pre-run-abe63578-86c8-43f0-b1c5-...,POWEROPS_SHOP_post-run-abe63578-86c8-43f0-b1c5...,POWEROPS_SHOP_shop-abe63578-86c8-43f0-b1c5-614...,POWEROPS_SHOP_cplex-abe63578-86c8-43f0-b1c5-61...,None,None,None,"{'version': 1, 'last_updated_time': 2024-10-21..."


Filter out just the results linked to `EXAMPLE_CASE_FORNEBU`, still as a dataframe

In [38]:
# Get the row where the case is EXAMPLE_CASE_B
case_fornebu_results: pd.DataFrame = all_results_df_2.loc[
    all_results_df_2["case"] == EXAMPLE_CASE_FORNEBU
]

case_fornebu_results

,space,external_id,case,objective_value,pre_run,post_run,messages,cplex_logs,alerts,output_time_series,node_type,data_record
1,power_ops_instances,shop_result__294f124d-0546-40b4-94fd-4d2a40194f32,example_case_fornebu,"{'total': -19927822739.279236, 'load_value': 0...",POWEROPS_SHOP_pre-run-294f124d-0546-40b4-94fd-...,POWEROPS_SHOP_post-run-294f124d-0546-40b4-94fd...,POWEROPS_SHOP_shop-294f124d-0546-40b4-94fd-4d2...,POWEROPS_SHOP_cplex-294f124d-0546-40b4-94fd-4d...,None,None,None,"{'version': 1, 'last_updated_time': 2024-10-21..."
2,power_ops_instances,shop_result__abe63578-86c8-43f0-b1c5-6143182916d2,example_case_fornebu,"{'total': -19927822739.279236, 'load_value': 0...",POWEROPS_SHOP_pre-run-abe63578-86c8-43f0-b1c5-...,POWEROPS_SHOP_post-run-abe63578-86c8-43f0-b1c5...,POWEROPS_SHOP_shop-abe63578-86c8-43f0-b1c5-614...,POWEROPS_SHOP_cplex-abe63578-86c8-43f0-b1c5-61...,None,None,None,"{'version': 1, 'last_updated_time': 2024-10-21..."


Convert to instances

In [39]:
shop_results_fornebu: ShopResultList = result_instance_retrieve(case_fornebu_results.external_id)
shop_results_fornebu

,space,external_id,case,objective_value,pre_run,post_run,messages,cplex_logs,alerts,output_time_series,node_type,data_record
0,power_ops_instances,shop_result__294f124d-0546-40b4-94fd-4d2a40194f32,example_case_fornebu,"{'total': -19927822739.279236, 'load_value': 0...",POWEROPS_SHOP_pre-run-294f124d-0546-40b4-94fd-...,POWEROPS_SHOP_post-run-294f124d-0546-40b4-94fd...,POWEROPS_SHOP_shop-294f124d-0546-40b4-94fd-4d2...,POWEROPS_SHOP_cplex-294f124d-0546-40b4-94fd-4d...,[shop_penalty_report_b_dummy_scenario_2023-01-...,None,None,"{'version': 1, 'last_updated_time': 2024-10-21..."
1,power_ops_instances,shop_result__abe63578-86c8-43f0-b1c5-6143182916d2,example_case_fornebu,"{'total': -19927822739.279236, 'load_value': 0...",POWEROPS_SHOP_pre-run-abe63578-86c8-43f0-b1c5-...,POWEROPS_SHOP_post-run-abe63578-86c8-43f0-b1c5...,POWEROPS_SHOP_shop-abe63578-86c8-43f0-b1c5-614...,POWEROPS_SHOP_cplex-abe63578-86c8-43f0-b1c5-61...,[shop_penalty_report_b_dummy_scenario_2023-01-...,None,None,"{'version': 1, 'last_updated_time': 2024-10-21..."


In [40]:
from datetime import timezone  # Data Record last updated time is in UTC

for result in shop_results_fornebu:
    updated_time_utc = result.data_record.last_updated_time.replace(
        tzinfo=timezone.utc
    ).strftime(
        "%Z: %a %d %b %Y, %H:%M:%S",
    )
    print(
        f"Result with external ID {result.external_id} was last updated at {updated_time_utc}"
    )

Result with external ID shop_result__294f124d-0546-40b4-94fd-4d2a40194f32 was last updated at UTC: Mon 21 Oct 2024, 08:30:05
Result with external ID shop_result__abe63578-86c8-43f0-b1c5-6143182916d2 was last updated at UTC: Mon 21 Oct 2024, 08:34:15


## Cleaning up the instances that were created

Instances can be deleted by using the powerops client and the `external_id`s of everything we have created 

In [42]:
shop_case_external_ids = [EXAMPLE_CASE_STAVANGER, EXAMPLE_CASE_FORNEBU]
shop_result_external_ids = all_results_df_2.external_id.to_list()
all_external_ids = shop_case_external_ids + shop_result_external_ids

all_external_ids

['example_case_stavanger',
 'example_case_fornebu',
 'shop_result__dfd0ed6a-d975-4296-9b75-0574394e15f4',
 'shop_result__294f124d-0546-40b4-94fd-4d2a40194f32',
 'shop_result__abe63578-86c8-43f0-b1c5-6143182916d2']

In [43]:
powerops.v1.delete(external_id=all_external_ids)

InstancesDeleteResult(nodes=[NodeId(space='power_ops_instances', external_id='example_case_stavanger'), NodeId(space='power_ops_instances', external_id='example_case_fornebu'), NodeId(space='power_ops_instances', external_id='shop_result__dfd0ed6a-d975-4296-9b75-0574394e15f4'), NodeId(space='power_ops_instances', external_id='shop_result__294f124d-0546-40b4-94fd-4d2a40194f32'), NodeId(space='power_ops_instances', external_id='shop_result__abe63578-86c8-43f0-b1c5-6143182916d2')], edges=[])